<a href="https://colab.research.google.com/github/ad17171717/YouTube-Tutorials/blob/main/Google%20Colab%20Tutorials/Can't_Run_Docker_in_Google_Colab!_Try_udocker!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Why Docker Engine Does Not Work in Google Colab (for now)**

**Docker Engine is an open-source containerization technology for building and containerizing applications. To build and manage Docker containers, the user needs kernel-level access and full control of system resources. Full control is necessary because Docker relies on Linux kernel features such as Control Groups (cgroups) and Namespaces to create and manage containers.**

- **cgroups provide a mechanism in Linux for managing system resources. cgroups enable fine-grained control of resources such as CPU, memory, block I/O, and network bandwidth for Linux containers. Docker uses cgroups to allocate resources to containers, ensuring that each container operates within its defined resource limits without negatively impacting other containers or processes on the host system.**

- **Namespaces are used to isolate parts of the operating system. When a Docker container is started, the Docker engine instructs the kernel to create namespaces for the container. These namespaces ensure that different aspects of the container, such as networking, file systems, and process IDs (PIDs), are restricted to the container’s isolated environment. As a result, processes and resources inside one container are not visible to others, nor to the host system, unless explicitly shared. This isolation can be configured to allow certain shared resources or interactions, but by default, namespaces provide strong separation.**

**In the current Google Colab environment, users cannot configure or edit cgroups or Namespaces. This restriction exists because Google Colab runs in a shared environment where users do not have kernel-level privileges. Without control over these features, it is not possible to create or run Docker containers. As of October 2024, Google Colab does not support running Docker Engine. In the future, changes to Colab’s environment might allow users to run containers, but for now, these features remain unavailable.**

<sup>Source: [What is Docker?](https://docs.docker.com/get-started/docker-overview/) from docs.docker.com</sup>

<sup>Source: [Runtime metrics](https://docs.docker.com/engine/containers/runmetrics/) from docs.docker.com</sup>

<sup>Source: [CGROUPS](https://www.kernel.org/doc/Documentation/cgroup-v1/cgroups.txt) from kernel.org</sup>

### **Control Group (cgroup) limitations**

**In a Google Colab session, it is possible to create a cgroup; however, there are limited privileges to fully manage or modify cgroups.**

**Even though the default user in a Google Colab session is the `root` user, the `root` user’s permissions are restricted. Specifically, `root` does not have full read-write permissions to the `cgroup` directory. Attempts to modify the cgroup, such as writing to resource management files are denied due to insufficient privileges.**

**As a result, Docker, which relies on cgroups to manage resources for containers, cannot function properly in this environment if a cgroup cannot be modified.**

In [ ]:
#check the current user
!whoami

In [ ]:
#check that cgroups are limited to read only "ro"
!mount | grep /sys

In [ ]:
#attempt to remount with write permissions
!mount -o remount,rw /sys
!mount -o remount,rw /sys/fs/cgroup

In [ ]:
#check cgroup again for write privelages "rw"
!mount | grep /sys

In [ ]:
#create a new cgroup
!mkdir /sys/fs/cgroup/my_cgroup
!ls /sys/fs/cgroup

In [ ]:
#attempt to modify the cgroup
!echo 512 > /sys/fs/cgroup/my_cgroup/cpu.shares

## **Namespace limitations**

**Similar to cgroups, within a Google Colab session, namespaces can be created, but there is a lack of privileges to modify and manage those namespaces fully. For example, a network namespace can be created using the `unshare` command. However, when attempting to bring the network interface up (such as the loopback interface) using `ip link set lo up`, we receive an error stating that the operation is not permitted. This limitation arises because the necessary administrative capabilities are missing. Docker requires the ability to manage and control network interfaces within network namespaces to allow containers to communicate with the host and other containers. Without the ability to perform these operations, Docker cannot function properly in this environment.**

In [ ]:
#create a new network namespace
!unshare --net bash -c "ip link"

In [ ]:
#attempt to bring the newly created network namespace up
!ip link set lo up

## **Attempting to Install Docker**

In [ ]:
!sudo apt update
!sudo apt install docker.io

In [ ]:
#attempt to start Docker
!service docker start

In [ ]:
#attempt to pull/run the Docker hello-world image
!docker run hello-world

# **udocker**

**An alternative to Docker is udocker. udocker is a user-space tool that enables running Docker containers in environments where users lack root privileges, such as Linux batch systems, HPC clusters and interactive systems. It allows non-privileged users to download, create, and execute containers without admin privileges. udocker acts as a wrapper around various technologies like PRoot, Fakechroot, runc, and Singularity to mimic basic Docker functionalities. Unlike Docker, which relies on kernel-level features such as namespaces and cgroups for isolation and resource management, udocker simulates root privileges using user namespaces and ptrace system calls via PRoot or Fakechroot. However, udocker does not offer the robust container isolation or system-level integration that Docker provides, making it suitable for environments where Docker cannot be used but with certain security and functionality limitations.**

<sup>Source: [udocker user manual Documentation](https://indigo-dc.github.io/udocker/user_manual.html)</sup>

## **Installing udocker**

**One way to install udocker is with the `pip` package manager for Python**

<sup>Source: [udocker Installation Documentation](https://indigo-dc.github.io/udocker/installation_manual.html)</sup>

In [ ]:
!pip install udocker

## **Pulling an Image with udocker**

**By default udocker pulls images from Dockerhub. In this section we will pull the [Golang (Go)](https://hub.docker.com/_/golang) image from Dockerhub and run a simple Go script that will print "Hello World" using udocker.**

In [ ]:
!whoami

In [ ]:
#cannot run udocker as root
!udocker pull golang

In [ ]:
#create non-root user to run udocker
!adduser --disabled-password --gecos '' newuser

In [ ]:
#pull a Docker image to work with
!sudo -u newuser udocker pull golang

## **Running udocker**

In [ ]:
#check go version within the go "container"
!sudo -u newuser udocker run golang go version

In [ ]:
#create a simple Go script
%%bash
cat > hello.go
package main

import "fmt"

func main() {
    fmt.Println("Hello, World!")
}

In [ ]:
#run the Go script with udocker
!sudo -u newuser udocker run --volume=$(pwd):/workspace -w=/workspace golang go run hello.go

## **Issues with udocker in Google Colab**

**One of the issues with running udocker in Google Colab is that it can be slow. For example, printing "Hello World" with udocker and Go takes 40 seconds on average, while printing "Hello World" with the Go binary in Google Colab takes 250 milliseconds on average. This makes Go installed directly more than 1,000 times faster than when run through udocker. The reason for this slowdown is the emulation layer used by udocker, which is inherently slower than if udocker had direct system access.**

**To work with udocker, a non-root user needs to be created. The commands to run a udocker container can be relatively verbose compared to Docker commands run in a dedicated machine.**

**Because networking within Google Colab is restricted, there are images that cannot be utilized within Google Colab. For example the image for [Jupyter Notebook](https://hub.docker.com/r/jupyter/datascience-notebook) cannot be run because it runs a web server and requires opening ports for the notebook, which is not allowed in a Google Colab session.**

# **References and Additional Learning**

## **Docker Image**

- **[Golang Image](https://hub.docker.com/_/golang) from Dockerhub**

## **Documentation**

- **[CGROUPS](https://www.kernel.org/doc/Documentation/cgroup-v1/cgroups.txt) from kernel.org**

- **[Docker Engine overview](https://docs.docker.com/engine/) from docs.docker.com**

- **[dockerd](https://docs.docker.com/reference/cli/dockerd/) from docs.docker.com**

- **[Runtime metrics](https://docs.docker.com/engine/containers/runmetrics/) from docs.docker.com**

- **[udocker Installation Documentation](https://indigo-dc.github.io/udocker/installation_manual.html)**

- **[udocker user manual Documentation](https://indigo-dc.github.io/udocker/user_manual.html)**

- **[What is Docker?](https://docs.docker.com/get-started/docker-overview/) from docs.docker.com**

# **Connect**
- **Feel free to connect with Adrian on [YouTube](https://www.youtube.com/channel/UCPuDxI3xb_ryUUMfkm0jsRA), [LinkedIn](https://www.linkedin.com/in/adrian-dolinay-frm-96a289106/), [X](https://twitter.com/DolinayG), [GitHub](https://github.com/ad17171717), [Medium](https://adriandolinay.medium.com/) and [Odysee](https://odysee.com/@adriandolinay:0). Happy coding!**

# **Podcast**

- **Check out Adrian's Podcast, The Aspiring STEM Geek on [YouTube](https://www.youtube.com/@AdrianDolinay/podcasts), [Spotify](https://open.spotify.com/show/60dPNJbDPaPw7ru8g5btxV), [Apple Podcasts](https://podcasts.apple.com/us/podcast/the-aspiring-stem-geek/id1765996824), [Audible](https://www.audible.com/podcast/The-Aspiring-STEM-Geek/B0DC73S9SN?eac_link=MCFKvkxuqKYU&ref=web_search_eac_asin_1&eac_selected_type=asin&eac_selected=B0DC73S9SN&qid=IrZ84nGqvz&eac_id=141-8769271-5781515_IrZ84nGqvz&sr=1-1) and [iHeart Radio](https://www.iheart.com/podcast/269-the-aspiring-stem-geek-202676097/)!**